In [1]:
%matplotlib inline

In [2]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt

from IPython.display import clear_output
from torch.utils.data import DataLoader, TensorDataset
from torchsummary import summary
from sklearn.model_selection import train_test_split

from data_generator import DataGen

In [35]:
class MMC(nn.Module):
    
    def __init__(self, beta=5, num_iter=20):
        super(MMC, self).__init__()
        self.df = 1. / beta
        self.num_iter = num_iter
        # self.weights = nn.Parameter(data= torch.tensor([[1, self.df, 0], [-1, 0, 1], [0, 0, 1]]),
        #                            requires_grad=False)
        self.weights = nn.Parameter(data=torch.from_numpy(self._init_weights()).float(), 
                                    requires_grad=False)
    
    def _init_weights(self):
        w = np.array([[1, self.df, 0], [-1, 0, 1], [0, 0, 1]])
        return np.broadcast_to(w, (9, 3, 3))
        
    def forward(self, x):
        x = torch.bmm(torch.pow(self.weights, self.num_iter), x.unsqueeze(2)).squeeze(2)
        # x = torch.matmul(self.weights, x.transpose_(0, 1))
        return x

In [36]:
class Encoder(nn.Module):
    
    def __init__(self, num_channel):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=num_channel, out_channels=16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 8, 3, padding=1)
        self.conv3 = nn.Conv2d(8, 8, 3, padding=1)
        self.conv4 = nn.Conv2d(8, 8, 2)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(200, 128)
        self.fc2 = nn.Linear(128, 3)
        self.mmc = MMC()
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))
        x = x.view(-1, 5*5*8)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        # print('before: ', x)
        x = self.mmc(x)
        # print('after: ', x)
        return x

In [37]:
model = Encoder(1)
for n, p in model.named_parameters():
    print(n, p.shape)

conv1.weight torch.Size([16, 1, 3, 3])
conv1.bias torch.Size([16])
conv2.weight torch.Size([8, 16, 3, 3])
conv2.bias torch.Size([8])
conv3.weight torch.Size([8, 8, 3, 3])
conv3.bias torch.Size([8])
conv4.weight torch.Size([8, 8, 2, 2])
conv4.bias torch.Size([8])
fc1.weight torch.Size([128, 200])
fc1.bias torch.Size([128])
fc2.weight torch.Size([3, 128])
fc2.bias torch.Size([3])
mmc.weights torch.Size([9, 3, 3])


In [38]:
# summary(model, (1, 100, 100))

In [39]:
def get_model(num_channel):
    
    net = Encoder(num_channel=num_channel)
    optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
    
    return net, optimizer

In [40]:
def to_tensor(x):
    return torch.from_numpy(x).float()

def get_data(height, width, num_channel, path, size, bs):
    
    dgen = DataGen(height, width, num_channel)
    x, y = dgen.get_data(path, True, size, True)
    print('Data loaded...\nx:{}\ty:{}\n'.format(x.shape, y.shape))
    
    x = x / 255.
    x = x[:90]
    y = y[:90]
    
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, shuffle=True)
    x_train, x_test, y_train, y_test = map(to_tensor, (x_train, x_test, y_train, y_test))
    
    train_ds = TensorDataset(x_train, y_train)
    train_dl = DataLoader(train_ds, batch_size=bs, shuffle=True)

    test_ds = TensorDataset(x_test, y_test)
    test_dl = DataLoader(test_ds, batch_size=bs)
    
    return train_dl, test_dl

In [41]:
def grad_flow(named_parameters):
    ave_grads = []
    max_grads= []
    layers = []
    for n, p in named_parameters:
        if(p.requires_grad) and ("bias" not in n):
            layers.append(n)
            ave_grads.append(p.grad.abs().mean())
            max_grads.append(p.grad.abs().max())
    print('layers: {}'.format(layers))
    print('max: {}'.format(max_grads))
    print('mean: {}\n\n'.format( ave_grads))

In [42]:
def fit(net, optimizer, train_dl, test_dl, epochs):
    
    loss_function = nn.MSELoss()
    for epoch in range(epochs):
        net.train()
        for x, y in train_dl:
            pred = net(x)
            loss = loss_function(pred, y)

            optimizer.zero_grad()
            loss.backward()
            # grad_flow(net.named_parameters())
            optimizer.step()

        net.eval()
        with torch.no_grad():
            test_loss = sum(loss_function(net(x), y) for x, y in test_dl)
        print('epoch:{}, test_loss:{}'.format(epoch+1, test_loss/len(test_dl)))

In [43]:
DATA_DIR = './data/data_simple_movement/cartesian/'
HEIGHT = 100
WIDTH = 100
NUM_CHANNEL = 2

In [44]:
net, optimizer = get_model(num_channel=2)

train_dl, test_dl = get_data(
    height=HEIGHT, 
    width=WIDTH,
    num_channel=NUM_CHANNEL,
    path=DATA_DIR,
    size=8,
    bs=9
)

fit(net, optimizer, train_dl, test_dl, epochs=100)

Data loaded...
x:(96, 2, 100, 100)	y:(96, 3)

epoch:1, test_loss:0.16507582366466522
epoch:2, test_loss:0.1658763438463211
epoch:3, test_loss:0.16412250697612762
epoch:4, test_loss:0.16304783523082733
epoch:5, test_loss:0.16334226727485657
epoch:6, test_loss:0.1647080034017563
epoch:7, test_loss:0.16266800463199615
epoch:8, test_loss:0.16278061270713806
epoch:9, test_loss:0.1632707118988037
epoch:10, test_loss:0.15794607996940613
epoch:11, test_loss:0.15149512887001038
epoch:12, test_loss:0.15696869790554047
epoch:13, test_loss:0.14606626331806183
epoch:14, test_loss:0.1465470790863037
epoch:15, test_loss:0.16499516367912292
epoch:16, test_loss:0.15720269083976746
epoch:17, test_loss:0.12926284968852997
epoch:18, test_loss:0.12916569411754608
epoch:19, test_loss:0.14382973313331604
epoch:20, test_loss:0.12743353843688965
epoch:21, test_loss:0.1272801160812378
epoch:22, test_loss:0.12498456239700317
epoch:23, test_loss:0.13436101377010345
epoch:24, test_loss:0.12304960191249847
epoch:25

In [45]:
for x, y in test_dl:
    print('prediction: {}, target: {}'.format(net(x), y))

prediction: tensor([[ 0.1777,  0.3377,  0.1600],
        [ 0.2895,  0.5750,  0.2855],
        [-0.2511, -0.5110, -0.2599],
        [-0.1816, -0.3755, -0.1940],
        [ 0.0409,  0.0696,  0.0288],
        [ 0.1577,  0.3003,  0.1425],
        [-0.0832, -0.1816, -0.0984],
        [ 0.0843,  0.1548,  0.0705],
        [-0.0112, -0.0354, -0.0242]], grad_fn=<SqueezeBackward1>), target: tensor([[ 5.0227e-01,  3.6272e-03,  5.0000e-01],
        [ 8.6668e-01,  1.8136e-03,  8.6603e-01],
        [-8.6273e-01,  9.0679e-03, -8.6603e-01],
        [-2.5438e-01,  6.3475e-03, -2.5882e-01],
        [ 3.9369e-03,  5.4407e-03,  6.1232e-17],
        [ 5.0227e-01,  3.6272e-03,  5.0000e-01],
        [-2.5438e-01,  6.3475e-03, -2.5882e-01],
        [ 2.6199e-01,  4.5339e-03,  2.5882e-01],
        [ 3.9369e-03,  5.4407e-03,  6.1232e-17]])


In [11]:
a = np.array([[-0.0472], [-0.0123],  [0.0237]])
b = np.array([[1, 0.2, 0], [-1, 0, 1], [0, 0, 1]])
np.dot(b, a)

array([[-0.04966],
       [ 0.0709 ],
       [ 0.0237 ]])

In [11]:
#         iteration = 0
#         while iteration < self.num_iter:
#             for i in range(x.shape[0]):
#                 x[i] = torch.matmul(self.weights, x[i])
#             iteration += 1
        # x = torch.acos(x)

In [20]:
a = torch.tensor([[1, 2, 3]])
a

tensor([[1, 2, 3]])

In [21]:
a.transpose(0, 1)

tensor([[1],
        [2],
        [3]])